# Preprocessing

Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
torch.cuda.is_available()
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
from torchvision import datasets
import datetime

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:04<00:00, 41848233.90it/s]


Extracting ../data-unversioned/p1ch7/cifar-10-python.tar.gz to ../data-unversioned/p1ch7/
Files already downloaded and verified


# Neural Network

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 64 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [5]:
train_loader = DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

def validate(model, loader, device):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    model.train()
    return correct / total

In [6]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device):
    start_time = time.time()
    for epoch in range(1, n_epochs + 1):
        model.train()
        loss_train = 0.0
        start_time = time.time()

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        val_accuracy = validate(model, val_loader, device) *100
        avg_loss = loss_train / len(train_loader)
        print(f"Epoch: {epoch}, Training Loss: {avg_loss}")
        print(f'Validation Accuracy after epoch {epoch}: {val_accuracy}%')
    end_time = time.time()
    duration = end_time - start_time
    print(f'Total Training Time: {duration} seconds')

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [8]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.096739678279213
Validation Accuracy after epoch 1: 34.68%
Epoch: 2, Training Loss: 1.7680414412027734
Validation Accuracy after epoch 2: 40.06%
Epoch: 3, Training Loss: 1.6244157139602524
Validation Accuracy after epoch 3: 43.94%
Epoch: 4, Training Loss: 1.537195571090864
Validation Accuracy after epoch 4: 46.06%
Epoch: 5, Training Loss: 1.4704386471482493
Validation Accuracy after epoch 5: 48.44%
Epoch: 6, Training Loss: 1.4133844704884093
Validation Accuracy after epoch 6: 49.559999999999995%
Epoch: 7, Training Loss: 1.3514177011102058
Validation Accuracy after epoch 7: 53.33%
Epoch: 8, Training Loss: 1.2950311509697028
Validation Accuracy after epoch 8: 54.96%
Epoch: 9, Training Loss: 1.2419594458454406
Validation Accuracy after epoch 9: 50.56%
Epoch: 10, Training Loss: 1.1926414908655465
Validation Accuracy after epoch 10: 56.55%
Epoch: 11, Training Loss: 1.1482998403289435
Validation Accuracy after epoch 11: 58.4%
Epoch: 12, Training Loss: 1.107193526099

# adding one more additional convolution layer followed by an activation function and pooling function

In [29]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device):
    start_time = time.time()
    for epoch in range(1, n_epochs + 1):
        model.train()
        loss_train = 0.0
        start_time = time.time()

        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        val_accuracy = validate(model, val_loader, device) *100
        avg_loss = loss_train / len(train_loader)
        print(f"Epoch: {epoch}, Training Loss: {avg_loss}")
        print(f'Validation Accuracy after epoch {epoch}: {val_accuracy}%')
    end_time = time.time()
    duration = end_time - start_time
    print(f'Total Training Time: {duration} seconds')

In [32]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 128 * 4 * 4)
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out

In [33]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [34]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.202404294172516
Validation Accuracy after epoch 1: 25.790000000000003%
Epoch: 2, Training Loss: 1.9527263124580578
Validation Accuracy after epoch 2: 34.300000000000004%
Epoch: 3, Training Loss: 1.7619632994732284
Validation Accuracy after epoch 3: 40.839999999999996%
Epoch: 4, Training Loss: 1.6130900483607027
Validation Accuracy after epoch 4: 42.38%
Epoch: 5, Training Loss: 1.5178910172199045
Validation Accuracy after epoch 5: 47.53%
Epoch: 6, Training Loss: 1.4407366872443568
Validation Accuracy after epoch 6: 49.8%
Epoch: 7, Training Loss: 1.372091787702897
Validation Accuracy after epoch 7: 51.68000000000001%
Epoch: 8, Training Loss: 1.313078278165949
Validation Accuracy after epoch 8: 52.290000000000006%
Epoch: 9, Training Loss: 1.2568415203667662
Validation Accuracy after epoch 9: 55.559999999999995%
Epoch: 10, Training Loss: 1.2042441405451205
Validation Accuracy after epoch 10: 58.620000000000005%
Epoch: 11, Training Loss: 1.1530508490474634
Validat